# N body simulation

In [1]:
from IPython.core.display import HTML
css_file = 'https://raw.githubusercontent.com/ngcm/training-public/master/ipython_notebook_styles/ngcmstyle.css'
HTML(url=css_file)

In [2]:
import numpy
from matplotlib import pyplot
%matplotlib inline
from matplotlib import rcParams
rcParams['font.family'] = 'serif'
rcParams['font.size'] = 16

In [Greg's original post](http://software-carpentry.org/blog/2014/10/why-we-dont-teach-testing.html) he specifically references the [Python 3 version](http://benchmarksgame.alioth.debian.org/u32/program.php?test=nbody&lang=python3&id=1) of the [n-body benchmark](http://benchmarksgame.alioth.debian.org/u32/performance.php?test=nbody). In particular, he asks how to test the `advance` function. Let's reproduce that code here.

In [3]:
# The Computer Language Benchmarks Game
# http://benchmarksgame.alioth.debian.org/
#
# originally by Kevin Carson
# modified by Tupteq, Fredrik Johansson, and Daniel Nanz
# modified by Maciej Fijalkowski
# 2to3

import sys 

def combinations(l):
    result = []
    for x in range(len(l) - 1):
        ls = l[x+1:]
        for y in ls:
            result.append((l[x],y))
    return result

PI = 3.14159265358979323
SOLAR_MASS = 4 * PI * PI
DAYS_PER_YEAR = 365.24

BODIES = {
    'sun': ([0.0, 0.0, 0.0], [0.0, 0.0, 0.0], SOLAR_MASS),

    'jupiter': ([4.84143144246472090e+00,
                 -1.16032004402742839e+00,
                 -1.03622044471123109e-01],
                [1.66007664274403694e-03 * DAYS_PER_YEAR,
                 7.69901118419740425e-03 * DAYS_PER_YEAR,
                 -6.90460016972063023e-05 * DAYS_PER_YEAR],
                9.54791938424326609e-04 * SOLAR_MASS),

    'saturn': ([8.34336671824457987e+00,
                4.12479856412430479e+00,
                -4.03523417114321381e-01],
               [-2.76742510726862411e-03 * DAYS_PER_YEAR,
                4.99852801234917238e-03 * DAYS_PER_YEAR,
                2.30417297573763929e-05 * DAYS_PER_YEAR],
               2.85885980666130812e-04 * SOLAR_MASS),

    'uranus': ([1.28943695621391310e+01,
                -1.51111514016986312e+01,
                -2.23307578892655734e-01],
               [2.96460137564761618e-03 * DAYS_PER_YEAR,
                2.37847173959480950e-03 * DAYS_PER_YEAR,
                -2.96589568540237556e-05 * DAYS_PER_YEAR],
               4.36624404335156298e-05 * SOLAR_MASS),

    'neptune': ([1.53796971148509165e+01,
                 -2.59193146099879641e+01,
                 1.79258772950371181e-01],
                [2.68067772490389322e-03 * DAYS_PER_YEAR,
                 1.62824170038242295e-03 * DAYS_PER_YEAR,
                 -9.51592254519715870e-05 * DAYS_PER_YEAR],
                5.15138902046611451e-05 * SOLAR_MASS) }


SYSTEM = list(BODIES.values())
PAIRS = combinations(SYSTEM)


def advance(dt, n, bodies=SYSTEM, pairs=PAIRS):

    for i in range(n):
        for (([x1, y1, z1], v1, m1),
             ([x2, y2, z2], v2, m2)) in pairs:
            dx = x1 - x2
            dy = y1 - y2
            dz = z1 - z2
            mag = dt * ((dx * dx + dy * dy + dz * dz) ** (-1.5))
            b1m = m1 * mag
            b2m = m2 * mag
            v1[0] -= dx * b2m
            v1[1] -= dy * b2m
            v1[2] -= dz * b2m
            v2[0] += dx * b1m
            v2[1] += dy * b1m
            v2[2] += dz * b1m
        for (r, [vx, vy, vz], m) in bodies:
            r[0] += dt * vx
            r[1] += dt * vy
            r[2] += dt * vz

The first thing to note is the rules of the exercise, again. We're treating the function as a black box: we believe it should be implementing an algorithm with certain behaviour, and we want to check what that behaviour is. In this case it's using the [*semi-implicit Euler method*](http://en.wikipedia.org/wiki/Semi-implicit_Euler_method) (I think: would have been nice if this were documented!) to update in time, and a [direct particle-particle method](http://en.wikipedia.org/wiki/N-body_problem#Few_bodies) to compute the spatial interactions.

That is, we can think of the algorithm as solving the ODE

$$
\begin{equation}
  \frac{d y}{dt} = F(t, y)
\end{equation}
$$

where the vector $y$ contains the positions $x$ and velocities $v$ of each body. Due to the physics of the problem (ie, there's a total energy that we want to conserve) there's an advantage in considering the positions and velocities separately, but when it comes to measure the error we don't need to do that: we just treat the `advance` function as an algorithm that takes the full vector $y_n$ as input and returns $y_{n+1}$.

What this means in practical terms is that we expect the error in time to have similar convergence rate behaviour to the standard Euler method discussed before: the convergence rate should be $1$, which can be checked both by the self-convergence being within $0.585$ and $1.585$, and also that the error in the convergence rate goes down by at least $1/3$ as the base resolution of $\Delta t$ is reduced by $2$. Checking the local truncation error would require more detailed analysis, but in principle should follow the same steps as before.

What's more interesting here is the additional checks from the complexities of the particle-particle interaction. We can use our previous techniques to check that the time update algorithm is behaving as expected. However, the particle-particle interaction itself contains many steps, and those also need testing.

Let's check the output on (a copy of) the default input, taking a *single* step.

In [4]:
import copy
bodies = copy.deepcopy(SYSTEM)
advance(dt=0.01, n=1, bodies=bodies, pairs=combinations(bodies))
print(bodies)
print(SYSTEM)

[([1.5983797301314366e-07, -3.0187796864956455e-08, -3.730115986360403e-09], [1.5983797301314366e-05, -3.0187796864956456e-06, -3.7301159863604027e-07], 39.47841760435743), ([4.847339930856543, -1.1321630550460413, -0.10387091638393478], [0.5908488391821772, 2.8156988981387063, -0.02488719128116714], 0.03769367487038949), ([12.905190971986693, -15.102456648865521, -0.22341579268383283], [1.0821409847561863, 0.8694752833109706, -0.010821379117708814], 0.0017237240570597112), ([15.389485801827046, -25.913363875177772, 0.17891118741867673], [0.9788686976130451, 0.5950734810190818, -0.034758553169444574], 0.0020336868699246304), ([8.333218184594687, 4.1430349688596575, -0.40343728476075313], [-1.0148533649892928, 1.8236404735352374, 0.00861323535682711], 0.011286326131968767)]
[([0.0, 0.0, 0.0], [0.0, 0.0, 0.0], 39.47841760435743), ([4.841431442464721, -1.1603200440274284, -0.10362204447112311], [0.606326392995832, 2.81198684491626, -0.02521836165988763], 0.03769367487038949), ([12.8943695

First, let's create a utility function that will give the difference between two configurations of bodies.

In [34]:
def differences(bodies1, bodies2):
    """
    Compare two configurations.
    """
    assert len(bodies1) == len(bodies2), "Configurations must have same number of bodies! {}, {}".format(len(bodies1), len(bodies2))
    N = len(bodies1)
    d_positions = numpy.zeros((N, 3))
    d_velocities = numpy.zeros((N, 3))
    norm_difference = 0.0
    for n in range(N):
        d_positions[n, :] = numpy.array(bodies1[n][0]) - numpy.array(bodies2[n][0])
        d_velocities[n, :] = numpy.array(bodies1[n][1]) - numpy.array(bodies2[n][1])
        norm_difference += numpy.linalg.norm(d_positions[n, :]) + numpy.linalg.norm(d_velocities[n, :])
    return norm_difference, d_positions, d_velocities

In [35]:
norm1, d_x, d_v = differences(bodies, SYSTEM)

In [36]:
d_v

array([[  1.59837973e-05,  -3.01877969e-06,  -3.73011599e-07],
       [ -1.54775538e-02,   3.71205322e-03,   3.31170379e-04],
       [ -6.50021685e-04,   7.62265141e-04,   1.12582837e-05],
       [ -2.22034631e-04,   3.74482371e-04,  -2.59766537e-06],
       [ -4.07901881e-03,  -2.02189770e-03,   1.97473980e-04]])

Now we have to think about how to test the particle-particle interaction. The parameter we can change in order to do the test is the configuration `bodies` (and the associated list of `pairs`, which we'll always take to be `combinations(bodies)`).

So we can either change the positions, or the velocities, of the bodies, or their masses (or some combination of the three). What changes make meaningful tests?

Well, the coordinates chosen should not matter, *if* we change them in a self-consistent way, *and* interpret the results appropriately. For example, if we changed the direction of time (`dt` $\to$ `-dt`) and changed the sign of the velocities, then the results should be identical.

In [37]:
def flip_time(bodies):
    """
    Flip the time by flipping the velocity.
    """
    
    for i in range(len(bodies)):
        for j in range(3):
            bodies[i][1][j] *= -1.0

In [38]:
bodies_flip_time = copy.deepcopy(SYSTEM)
flip_time(bodies_flip_time)
advance(dt=-0.01, n=1, bodies=bodies_flip_time, pairs=combinations(bodies_flip_time))
flip_time(bodies_flip_time)
norm_flip_time, d_x_flip_time, d_v_flip_time = differences(bodies, bodies_flip_time)
print("Norm of differences is {}.".format(norm_flip_time))

Norm of differences is 0.0.


As expected, the positions are identical. Note that the velocities are significantly different, because they still have the opposite sign.

This is just an example of a conservation law resulting from a symmetry: the consequence of [Emmy Noether's famous theorem](http://en.wikipedia.org/wiki/Noether's_theorem). There is a [detailed list][1] of symmetries that a physical theory could have, each of which has an associated conservation law. Not all physical models obey all symmetries, and not every numerical  simulation obeys all the symmetries of the physical model (maybe due to boundary conditions, or numerical approximations). However, in the n-body case we expect a large number of these symmetries to hold, which we can check.

These include:

* discrete parity symmetry (sending $x \to -x$)
* discrete time symmetry (sending $t \to -t$)
* discrete charge symmetry (sending $m \to -m$: this makes no *physical* sense, but the theory still obeys it!)
* continuous space translation ($x \to x + a$)
* continuous space rotation
* continuous time translation (this is obeyed by the continuum model, but not by the numerical model)
* continuous conformal (like) symmetry ($t \to a t$, $x \to a x$, and $m \to a m$ all together).

[1]: (http://en.wikipedia.org/wiki/Symmetry_(physics)#Conservation_laws_and_symmetry)

Next, note that we should be able to flip individual coordinates: if $y \to -y$ and $v_y \to -v_y$, for example, then  the evolution of the other coordinates and velocities should be unaffected (and if this flip is reversed after evolution, the position should be identical).

In [39]:
def flip_coordinate(bodies, coord):
    """
    Flip a single coordinate direction
    """
    
    for i in range(len(bodies)):
        bodies[i][0][coord] *= -1.0
        bodies[i][1][coord] *= -1.0

In [40]:
bodies_flip_y = copy.deepcopy(SYSTEM)
flip_coordinate(bodies_flip_y, 1)
advance(dt=0.01, n=1, bodies=bodies_flip_y, pairs=combinations(bodies_flip_y))
flip_coordinate(bodies_flip_y, 1)
norm_flip_y, d_x_flip_y, d_v_flip_y = differences(bodies, bodies_flip_y)
print("Norm of differences is {}.".format(norm_flip_y))

Norm of differences is 0.0.


Next, note that only the *separation* between bodies is meant to matter. So if we arbitrarily translate all coordinates by some amount whilst leaving the velocities untouched, and reverse this after evolution, the result should be identical.

In [41]:
def translate_coordinate(bodies, shift):
    """
    Translate the coordinates of all bodies
    """
    
    for i in range(len(bodies)):
        for n in range(3):
            bodies[i][0][n] += shift[n]

In [42]:
shift = 10.0*(-0.5+numpy.random.rand(3)) # Random coordinate shift in [-5, 5]
bodies_shift = copy.deepcopy(SYSTEM)
translate_coordinate(bodies_shift, shift)
advance(dt=0.01, n=1, bodies=bodies_shift, pairs=combinations(bodies_shift))
translate_coordinate(bodies_shift, -shift)
norm_shift, d_x_shift, d_v_shift = differences(bodies, bodies_shift)
print("Norm of differences is {}.".format(norm_shift))

Norm of differences is 2.80385659842308e-15.


In this case the repeated operations introduce some floating point round-off error. This should be related to the round off introduced by the shift, and bounded by the number of bodies ($5$) multiplied by the total round-off introduced:

In [43]:
numpy.sum(numpy.abs(numpy.spacing(shift)))

1.1102230246251565e-15

The precise differences are:

In [44]:
print(d_x_shift)
print(d_v_shift)

[[ -6.62471880e-17  -1.58129980e-16  -2.14544258e-16]
 [  0.00000000e+00   0.00000000e+00   3.05311332e-16]
 [  0.00000000e+00  -1.77635684e-15   1.11022302e-16]
 [  0.00000000e+00   0.00000000e+00  -1.11022302e-16]
 [  0.00000000e+00   0.00000000e+00   3.33066907e-16]]
[[  0.00000000e+00   0.00000000e+00   6.35274710e-22]
 [  0.00000000e+00   0.00000000e+00   0.00000000e+00]
 [  0.00000000e+00   0.00000000e+00   0.00000000e+00]
 [  0.00000000e+00   0.00000000e+00   0.00000000e+00]
 [  0.00000000e+00   0.00000000e+00   0.00000000e+00]]


We see that the first and last bodies have errors around the expected floating point limit, and these dominate the resulting error (which is well within the expected bound).

Yet another coordinate transformation can be tried without affecting the results: a rotation. We'll pick three random Euler angles and [set up the rotation matrix](http://en.wikipedia.org/wiki/Rotation_matrix#In_three_dimensions). In general, we should do a translation first to ensure no body is at the origin (here, the Sun will be), to avoid specializing the problem, but given the generality of the code that's a very minor worry.

In [45]:
def rotate_bodies(bodies, angles, invert=False):
    """
    Rotate the coordinates of all bodies.
    """
    
    Rx = numpy.array([[1.0, 0.0, 0.0],
                      [0.0, numpy.cos(angles[0]), -numpy.sin(angles[0])],
                      [0.0, numpy.sin(angles[0]), numpy.cos(angles[0])]])
    Ry = numpy.array([[numpy.cos(angles[1]), 0.0, numpy.sin(angles[1])],
                      [0.0, 1.0, 0.0],
                      [-numpy.sin(angles[1]), 0.0, numpy.cos(angles[1])]])
    Rz = numpy.array([[numpy.cos(angles[2]), -numpy.sin(angles[2]), 0.0],
                      [numpy.sin(angles[2]), numpy.cos(angles[2]), 0.0],
                      [0.0, 0.0, 1.0]])
    if invert:
        R = numpy.dot(numpy.dot(Rx, Ry), Rz)
    else:
        R = numpy.dot(Rz, numpy.dot(Ry, Rx))
    for i in range(len(bodies)):
        x = numpy.array(bodies[i][0])
        v = numpy.array(bodies[i][1])
        xp = numpy.dot(R, x)
        vp = numpy.dot(R, v)
        for n in range(3):
            bodies[i][0][n] = xp[n]
            bodies[i][1][n] = vp[n]

In [46]:
angles = numpy.pi/4.0*numpy.random.rand(3) # Random Euler angles in [0, pi/4]
bodies_rotate = copy.deepcopy(SYSTEM)
rotate_bodies(bodies_rotate, angles)
advance(dt=0.01, n=1, bodies=bodies_rotate, pairs=combinations(bodies_rotate))
rotate_bodies(bodies_rotate, -angles, invert=True)
norm_rotate, d_x_rotate, d_v_rotate = differences(bodies, bodies_rotate)
print("Norm of differences is {}.".format(norm_rotate))

Norm of differences is 9.612914171481047e-15.


The size of the differences is considerably larger, thanks to the number of operations performed. With three rotation matrices combined, each containing nine entries, applied to five bodies, we would expect a total error of the order of:

In [47]:
3*9*5*numpy.sum(numpy.abs(numpy.spacing(angles)))

1.5456386170953351e-14

We could also try scaling the coordinates and the masses. As the [force goes as](http://en.wikipedia.org/wiki/N-body_problem#General_formulation) $M^2/L^2$, if we scale time, position and mass by the same amount then the results should not change. Note that because we scale *both* length and time, the velocity does not change.

In [48]:
def scale_bodies(bodies, scale):
    """
    Scale coordinates and masses.
    """
    
    bodies_scale = []
    for (x, v, m) in bodies:
        new_x = copy.deepcopy(x)
        new_v = copy.deepcopy(v)
        new_m = m * scale
        for i in range(3):
            new_x[i] *= scale
        bodies_scale.append((new_x, new_v, new_m))
    
    return bodies_scale

In [49]:
scale = 2.0
bodies_scale = scale_bodies(SYSTEM, scale)
advance(dt=0.02, n=1, bodies=bodies_scale, pairs=combinations(bodies_scale))
bodies_rescale = scale_bodies(bodies_scale, 1.0/scale)
norm_scale, d_x_scale, d_v_scale = differences(bodies, bodies_rescale)
print("Norm of differences is {}.".format(norm_scale))

Norm of differences is 0.0.
